In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, InterleavedQubitMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP, COBYLA, P_BFGS,ADAM,AQGD,NFT
from qiskit.primitives import Estimator

import scipy
import numpy as np
from math import comb

In [ ]:

a = 1.0
geom = f"H 0 0 0; H {a} 0 0; H {a*2} 0 0; H {a*3} 0 0; H {a*4} 0 0; H {a*5} 0 0; H {a*6} 0 0; H {a*7} 0 0"


print("Geometry: ", geom)
# unit = DistanceUnit.BOHR
unit = DistanceUnit.ANGSTROM
##
driver = PySCFDriver(
    atom=geom,
    basis="sto3g",
    charge=0,
    spin=0,
    unit=unit,
)
es_problem = driver.run()
mapper = JordanWignerMapper()



qiskit_ham = es_problem.hamiltonian
## Add Nuclear Repulsion to Hamiltonian
from qiskit_nature.second_q.operators import PolynomialTensor
qiskit_ham.electronic_integrals.alpha += PolynomialTensor({
    "": qiskit_ham.nuclear_repulsion_energy
})
qiskit_ham.nuclear_repulsion_energy = None
##
qiskit_ferm_ham = qiskit_ham.second_q_op()
qiskit_ham_mat = mapper.map(qiskit_ferm_ham).to_matrix(sparse=True)
true_lowest_ev = scipy.sparse.linalg.eigsh(qiskit_ham_mat,1,which='SA')[0][0].real
print("True Lowest Eigenvalue: ", true_lowest_ev)

Geometry:  H 0 0 0; H 1.0 0 0; H 2.0 0 0; H 3.0 0 0; H 4.0 0 0; H 5.0 0 0; H 6.0 0 0; H 7.0 0 0
True Lowest Eigenvalue:  -4.307571602006787


In [4]:
es_problem.num_spatial_orbitals, es_problem.num_particles,

(8, (4, 4))

In [ ]:
ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)
# ##
# vqe_solver = VQE(Estimator(), ansatz,  NFT(maxiter=1000, disp=True)) ## COBYLA(maxiter=2000, disp=True) # P_BFGS(maxfun=100, iprint=10) => 3m 21s, error 0.06611099896936112, NFT(maxiter=10, disp=True) => 50s,0.38112909977208664
# vqe_solver.initial_point = [0.0] * ansatz.num_parameters
# ##
# from qiskit_nature.second_q.algorithms import GroundStateEigensolver
# calc = GroundStateEigensolver(mapper, vqe_solver)
# res = calc.solve(es_problem)
# print(res)

In [4]:
# res.hartree_fock_energy
ansatz.num_parameters

26

In [8]:
ansatz.excitation_ops()

[FermionicOp({'+_0 -_2': 1j, '+_2 -_0': (-0-1j)}, num_spin_orbitals=8, ),
 FermionicOp({'+_0 -_3': 1j, '+_3 -_0': (-0-1j)}, num_spin_orbitals=8, ),
 FermionicOp({'+_1 -_2': 1j, '+_2 -_1': (-0-1j)}, num_spin_orbitals=8, ),
 FermionicOp({'+_1 -_3': 1j, '+_3 -_1': (-0-1j)}, num_spin_orbitals=8, ),
 FermionicOp({'+_4 -_6': 1j, '+_6 -_4': (-0-1j)}, num_spin_orbitals=8, ),
 FermionicOp({'+_4 -_7': 1j, '+_7 -_4': (-0-1j)}, num_spin_orbitals=8, ),
 FermionicOp({'+_5 -_6': 1j, '+_6 -_5': (-0-1j)}, num_spin_orbitals=8, ),
 FermionicOp({'+_5 -_7': 1j, '+_7 -_5': (-0-1j)}, num_spin_orbitals=8, ),
 FermionicOp({'+_0 +_1 -_2 -_3': 1j, '+_3 +_2 -_1 -_0': (-0-1j)}, num_spin_orbitals=8, ),
 FermionicOp({'+_0 +_4 -_2 -_6': 1j, '+_6 +_2 -_4 -_0': (-0-1j)}, num_spin_orbitals=8, ),
 FermionicOp({'+_0 +_4 -_2 -_7': 1j, '+_7 +_2 -_4 -_0': (-0-1j)}, num_spin_orbitals=8, ),
 FermionicOp({'+_0 +_5 -_2 -_6': 1j, '+_6 +_2 -_5 -_0': (-0-1j)}, num_spin_orbitals=8, ),
 FermionicOp({'+_0 +_5 -_2 -_7': 1j, '+_7 +_2 -_

In [5]:
n_occ = 6 // 2
n_vir = (6 - n_occ) 


## Single
n_single = 2* n_occ * n_vir
print("Single: ", n_single)

## Double
s2s = comb(n_vir, 2) * comb(n_occ, 2)
n_doubels = s2s*2 + (n_occ*n_vir)**2
print("Double: ", s2s, n_doubels)

print("Total: ", n_single + n_doubels)

Single:  18
Double:  9 99
Total:  117


In [6]:
n_occ*n_vir

9

In [7]:
# n_sp = 4
# count = -1
# # occupied = list(range(n_sp))
# # unoccupied = n_sp+np.array(list(range(n_sp)))
# occupied = [0,1,6,7]
# unoccupied = [2,3,4,5,8,9,10,11]
# my_exc_list = []
# for s in occupied:
#     for r in occupied:
#         for q in unoccupied:
#             for p in unoccupied:
#                 if p>q and r >s:
#                     count += 1
#                     my_exc_list.append( ( (s,r),(q,p) ) )
#                     print(f"{count} :: (({s},{r}),({q},{p}))")

# my_exc_list = np.sort(my_exc_list)
# for i in range(len(my_exc_list)):
#     print(f"{i} :: {my_exc_list[i]}")

In [56]:
count = -1
for i in ansatz.excitation_list:
    count += 1
    print(f"{count} :: {i}")

0 :: ((0,), (3,))
1 :: ((0,), (4,))
2 :: ((0,), (5,))
3 :: ((1,), (3,))
4 :: ((1,), (4,))
5 :: ((1,), (5,))
6 :: ((2,), (3,))
7 :: ((2,), (4,))
8 :: ((2,), (5,))
9 :: ((6,), (9,))
10 :: ((6,), (10,))
11 :: ((6,), (11,))
12 :: ((7,), (9,))
13 :: ((7,), (10,))
14 :: ((7,), (11,))
15 :: ((8,), (9,))
16 :: ((8,), (10,))
17 :: ((8,), (11,))
18 :: ((0, 1), (3, 4))
19 :: ((0, 1), (3, 5))
20 :: ((0, 2), (3, 4))
21 :: ((0, 2), (3, 5))
22 :: ((0, 6), (3, 9))
23 :: ((0, 6), (3, 10))
24 :: ((0, 6), (3, 11))
25 :: ((0, 7), (3, 9))
26 :: ((0, 7), (3, 10))
27 :: ((0, 7), (3, 11))
28 :: ((0, 8), (3, 9))
29 :: ((0, 8), (3, 10))
30 :: ((0, 8), (3, 11))
31 :: ((0, 1), (4, 5))
32 :: ((0, 2), (4, 5))
33 :: ((0, 6), (4, 9))
34 :: ((0, 6), (4, 10))
35 :: ((0, 6), (4, 11))
36 :: ((0, 7), (4, 9))
37 :: ((0, 7), (4, 10))
38 :: ((0, 7), (4, 11))
39 :: ((0, 8), (4, 9))
40 :: ((0, 8), (4, 10))
41 :: ((0, 8), (4, 11))
42 :: ((0, 6), (5, 9))
43 :: ((0, 6), (5, 10))
44 :: ((0, 6), (5, 11))
45 :: ((0, 7), (5, 9))
46 :

In [5]:
import openfermion as of
import scipy.sparse as ss
import scipy.linalg as la
import scipy
from produce_xacc_ham import *
from _gcim_utilis import parse_hamiltonian

# folder_path = '../example_hamiltonians/mz_tests/'
# mol_name = 'H618'
n_qubits = 12
# from _gcim_utilis import parse_hamiltonian
# loaded_fermi_ham = parse_hamiltonian(f'{folder_path}mzqis_{mol_name}.hamil', n_qubits//2, use_interleaved=True, return_type = 'of')

folder_path = '../example_hamiltonians/'
loaded_fermi_ham = parse_hamiltonian(f'{folder_path}H6-6orbitals-1-angstom', n_qubits//2, use_interleaved=True, return_type = 'of')
loaded_ham_mat = of.linalg.get_sparse_operator(loaded_fermi_ham, n_qubits).todense()


n_a = 3
n_b = 3
n_orb = 6
occupied_list = []
for i in range(n_a):
    occupied_list.append(i*2)
for i in range(n_b):
    occupied_list.append(i*2+1)
reference_ket = scipy.sparse.csc_matrix(of.jw_configuration_state(occupied_list, 2*n_orb)).transpose()

ss.linalg.eigsh(loaded_ham_mat,1,which='SA',v0=reference_ket.todense())[0][0]

-3.2360662982521475

In [6]:
-3.2360662982521475 - -3.2354496041602117

-0.0006166940919358765

In [7]:
-3.2360662982521475 - -3.2354509528732631

-0.0006153453788844132

In [78]:
folder_path = '../example_hamiltonians/mz_tests/'
file_name = 'H6_3au_DUCC3_6-electrons_6-Orbitals.out-xacc'
n_qubits = 12
from _gcim_utilis import parse_hamiltonian
loaded_fermi_ham = parse_hamiltonian(f'{folder_path}{file_name}', n_qubits//2, use_interleaved=True, return_type = 'of')
loaded_ham_mat = of.linalg.get_sparse_operator(loaded_fermi_ham, n_qubits)

n_a = 3
n_b = 3
n_orb = 6
occupied_list = []
for i in range(n_a):
    occupied_list.append(i*2)
for i in range(n_b):
    occupied_list.append(i*2+1)
reference_ket = scipy.sparse.csc_matrix(of.jw_configuration_state(occupied_list, 2*n_orb)).transpose()

ss.linalg.eigsh(loaded_ham_mat,1,which='SA',v0=reference_ket.todense())[0][0]

-3.1747723379966795

In [4]:
import numpy as np
n_orb = 4

counter = 0

print("Single Excitations")
for p in range(0,n_orb):
    pa = 2*p
    pb = 2*p+1

    for q in range(p,n_orb):
        qa = 2*q
        qb = 2*q+1
        
        print(f"{counter}, ({pa},{pb})({qa},{qb})")
        counter += 1

print("Double Excitations")
pq = -1
for p in range(0,n_orb):
    pa = 2*p
    pb = 2*p+1

    for q in range(p,n_orb):
        qa = 2*q
        qb = 2*q+1

        pq += 1

        rs = -1
        for r in range(0,n_orb):
            ra = 2*r
            rb = 2*r+1

            for s in range(r,n_orb):
                sa = 2*s
                sb = 2*s+1

                rs += 1

                if(pq > rs):
                    continue
                print(f"{counter},{counter+1} ({pa},{pb})({qa},{qb})({ra},{rb})({sa},{sb})")
                counter += 2

Single Excitations
0, (0,1)(0,1)
1, (0,1)(2,3)
2, (0,1)(4,5)
3, (0,1)(6,7)
4, (2,3)(2,3)
5, (2,3)(4,5)
6, (2,3)(6,7)
7, (4,5)(4,5)
8, (4,5)(6,7)
9, (6,7)(6,7)
Double Excitations
10,11 (0,1)(0,1)(0,1)(0,1)
12,13 (0,1)(0,1)(0,1)(2,3)
14,15 (0,1)(0,1)(0,1)(4,5)
16,17 (0,1)(0,1)(0,1)(6,7)
18,19 (0,1)(0,1)(2,3)(2,3)
20,21 (0,1)(0,1)(2,3)(4,5)
22,23 (0,1)(0,1)(2,3)(6,7)
24,25 (0,1)(0,1)(4,5)(4,5)
26,27 (0,1)(0,1)(4,5)(6,7)
28,29 (0,1)(0,1)(6,7)(6,7)
30,31 (0,1)(2,3)(0,1)(2,3)
32,33 (0,1)(2,3)(0,1)(4,5)
34,35 (0,1)(2,3)(0,1)(6,7)
36,37 (0,1)(2,3)(2,3)(2,3)
38,39 (0,1)(2,3)(2,3)(4,5)
40,41 (0,1)(2,3)(2,3)(6,7)
42,43 (0,1)(2,3)(4,5)(4,5)
44,45 (0,1)(2,3)(4,5)(6,7)
46,47 (0,1)(2,3)(6,7)(6,7)
48,49 (0,1)(4,5)(0,1)(4,5)
50,51 (0,1)(4,5)(0,1)(6,7)
52,53 (0,1)(4,5)(2,3)(2,3)
54,55 (0,1)(4,5)(2,3)(4,5)
56,57 (0,1)(4,5)(2,3)(6,7)
58,59 (0,1)(4,5)(4,5)(4,5)
60,61 (0,1)(4,5)(4,5)(6,7)
62,63 (0,1)(4,5)(6,7)(6,7)
64,65 (0,1)(6,7)(0,1)(6,7)
66,67 (0,1)(6,7)(2,3)(2,3)
68,69 (0,1)(6,7)(2,3)(4,5)
70,71 (0,1)(

In [9]:
import numpy as np

n = 4
N1 = n * (n-1) * (n-2) * (n-3)
N2 = n*(n-1)*(4*n-6)
N3 = n * (n-1) * (n-2)
N4 = n * (n-1)
N5 = n * (n-1)
n**4, N1, N2, N3, N4, N5, N1+N2+N3+N4+N5

(256, 24, 120, 24, 12, 12, 192)